# NOTAS
###### O ENUNCIADO DIZIA PARA CONSIDERAR OS ULTIMOS 6 MESES DE CADA USUARIO ENTAO ME BASEEI NOS ULTIMOS 6 MESES DE CADA USUARIO COM BASE NAS DATAS DA BASE

## BIBLIOTECAS

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window

import sys

## LER TABELAS

In [0]:
df_users_yt = spark.table("workspace.default.users_yt")

df_posts_creator = spark.table("workspace.default.posts_creator")

## TABELA DE APOIO PARA ANALISES

In [0]:
df_analytics = df_users_yt.join(df_posts_creator, df_users_yt['user_id'] == df_posts_creator['yt_user'], 'left')

## ULTIMOS 6 MESES POR USER ID

In [0]:
w_max = Window.partitionBy("user_id")

df_with_max = df_analytics.withColumn("max_date", F.max("published_date").over(w_max))

df_with_limit = df_with_max.withColumn("limit_date", F.add_months("max_date", -5))

df_last_six_months = df_with_limit.filter(F.col("published_date") >= F.col("limit_date"))

df_last_six_months = df_last_six_months.withColumn('published_year', F.date_format(F.col('published_date'), 'yyyy'))\
                                       .withColumn('published_month', F.date_format(F.col('published_date'), 'MM'))\
                                       .withColumn("published_year_month", F.date_format(F.col("published_date"), "yyyy-MM"))

df_last_six_months.display()

## RANK LIKES

In [0]:
cols = [
    'user_id', 'title', 'likes', 'rank'
]

window1 = Window.partitionBy('user_id').orderBy(F.desc_nulls_last("likes"))

df_analytics01 = df_last_six_months.withColumn('rank', F.row_number().over(window1))\
                                   .filter(F.col('rank') <= 3)\
                                   .select(cols)

df_analytics01.display()

## RANK VIEWS

In [0]:
cols = ['user_id', 'title', 'views', 'rank']

w_rank = Window.partitionBy("user_id").orderBy(F.desc_nulls_last("views"))

df_analytics02 = df_last_six_months.withColumn("rank", F.row_number().over(w_rank))\
                                   .filter(F.col("rank") <= 3)\
                                   .select(cols)\
                                       
df_analytics02.display()

## LEFT ANTI YT USERS

In [0]:
df_leftant = df_posts_creator.join(df_users_yt, df_posts_creator['yt_user'] == df_users_yt['user_id'], 'leftanti').select('yt_user').distinct()
df_leftant.display()

## CONTAR POTS POR YT_USER

In [0]:
df_count_pots = df_last_six_months.groupBy('yt_user', 'published_year_month').count().orderBy('yt_user', 'published_year_month')
df_count_pots.display()

## GERACAO DE SERIE MENSAL POR USUARIO

In [0]:
# A FUNCAO TEM COMO OBJETIVO A PARTIR DOS VALORES MINIMOS E MAXIMOS DE PERIODO ANO E MES 
# GERAR TODOS OS MESES DENTRO DESSE INTERVALO PARA CADA YT_USER PREENCHENDO OS VALORES AUSENTES
def mul_valor_data(df):
    dfs = []
    for i in df.select('yt_user').distinct().collect():
        df_support = df.filter(F.col('yt_user')==i.yt_user)
        
        min_year = int(df_support.agg({"published_year_month": "min"}).collect()[0][0][:4])
        max_year =  int(df_support.agg({"published_year_month": "max"}).collect()[0][0][:4])

        meses_anos = [f"{ano}-{str(m).zfill(2)}"
                    for ano in range(min_year, max_year + 1)
                    for m in range(1, 13)]

        df_ano_mes = spark.createDataFrame(meses_anos, ['published_year_month'])

        df_support = df_ano_mes.join(df_support, on='published_year_month', how='left')

        # ERRO TRECHO E RESPONSAVEL POR PREENCHER OS VALORES NULOS COM VALOR DA LINHA ANTERIOR
        w = Window.orderBy(F.desc('yt_user')).rowsBetween(-sys.maxsize, 0)

        df_support = df_support.withColumn('yt_user_filled', F.last('yt_user', ignorenulls=True).over(w))\
                               .drop("yt_user")\
                               .withColumnRenamed('yt_user_filled', 'yt_user')\
                               .fillna(0, subset=['count'])
        
        dfs.append(df_support)
    
    df_geral = dfs[0]
    for df in dfs[1:]:
        df_geral = df_geral.union(df)
    
    return df_geral

df_full_months = mul_valor_data(df_count_pots).orderBy('yt_user', 'published_year_month')

df_full_months.display()

## PIVOT POR ANO - MES

In [0]:
df_pivot_months = df_full_months.groupBy("yt_user").pivot("published_year_month").agg(F.first("count"))

df_pivot_months.display()

## TOP 3 TAGS

In [0]:
df_tags = df_last_six_months.withColumn("tag", F.explode("tags"))

df_count_tags = df_tags.groupBy("yt_user", "tag")\
                       .count()

w = Window.partitionBy("yt_user").orderBy(F.desc("count"))
df_ranked = df_count_tags.withColumn("rank", F.row_number().over(w))

df_top3_tags = df_ranked.filter(F.col("rank") <= 3)

df_top3_tags.display()